In [ ]:
"""
SQL Practice
Part 2: Baseball Data

"""

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('lahman.sqlite')

query = "SELECT * FROM sqlite_master;"

df_schema = pd.read_sql_query(query, conn)

df_schema.tbl_name.unique()

array(['allstarfull', 'appearances', 'awardsmanagers', 'awardsplayers',
       'awardssharemanagers', 'awardsshareplayers', 'batting',
       'battingpost', 'collegeplaying', 'divisions', 'fielding',
       'fieldingof', 'fieldingofsplit', 'fieldingpost', 'halloffame',
       'homegames', 'leagues', 'managers', 'managershalf', 'parks',
       'people', 'pitching', 'pitchingpost', 'salaries', 'schools',
       'seriespost', 'teams', 'teamsfranchises', 'teamshalf'],
      dtype=object)

#### 1. What was the total spent on salaries by each team, each year?

In [2]:
query = """
SELECT teamID, team_ID, SUM(salary)
FROM salaries
GROUP BY teamID, yearID
"""
df_salaries = pd.read_sql_query(query, conn)
df_salaries

,teamID,team_ID,SUM(salary)
0,ATL,1918,14807000.0
1,BAL,1919,11560712.0
2,BOS,1920,10897560.0
3,CAL,1921,14427894.0
4,CHA,1922,9846178.0
...,...,...,...
913,SLN,2831,143053500.0
914,TBA,2832,57097310.0
915,TEX,2833,176038723.0
916,TOR,2834,138701700.0


#### 2. What is the first and last year played for each player? Hint: Create a new table from 'Fielding.csv'.

In [3]:
query = """
SELECT DISTINCT playerID,
        MIN(yearID) OVER (PARTITION BY playerID) AS "First Year",
        MAX(yearID) OVER (PARTITION BY playerID) AS "Last Year"
FROM fielding
GROUP BY playerID, yearID
"""
df = pd.read_sql_query(query, conn)
df

,playerID,First Year,Last Year
0,aardsda01,2004,2015
1,aaronha01,1954,1976
2,aaronto01,1962,1971
3,aasedo01,1977,1990
4,abadan01,2001,2003
...,...,...,...
19486,zupofr01,1957,1961
19487,zuvelpa01,1982,1991
19488,zuverge01,1951,1959
19489,zwilldu01,1910,1916


#### 3. Who has played the most all star games?

In [4]:
query = """
SELECT playerID, COUNT(ID)
FROM allstarfull
GROUP BY playerID
ORDER BY COUNT(ID) DESC
"""
df = pd.read_sql_query(query, conn)
df

,playerID,COUNT(ID)
0,aaronha01,24
1,mayswi01,24
2,musiast01,24
3,mantlmi01,19
4,ripkeca01,19
...,...,...
1862,zachrpa01,1
1863,zakfr01,1
1864,zarilal01,1
1865,zernigu01,1


#### 4. Which school has generated the most distinct players? Hint: Create new table from 'CollegePlaying.csv'.

In [5]:
query = """
SELECT name_full AS "School", COUNT(DISTINCT playerID) AS "Num of Players"
FROM collegeplaying
JOIN schools
ON schools.schoolID = collegeplaying.schoolID
GROUP BY name_full
ORDER BY COUNT(playerID) DESC
"""
df = pd.read_sql_query(query, conn)
df

,School,Num of Players
0,University of Texas at Austin,107
1,University of Southern California,105
2,Stanford University,86
3,Arizona State University,101
4,University of Michigan,76
...,...,...
1026,Blackburn College,1
1027,Barstow Community College,1
1028,Baltimore City Community College,1
1029,Augustana College,1


#### 5. Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. Hint: Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the DATE function and can then be subtracted from each other yielding their difference in days.

In [6]:
query = """
SELECT playerID, DATE(finalgame_date) - DATE(debut_date) AS "Career (years)"
FROM people
GROUP BY playerID
ORDER BY (DATE(finalgame_date) - DATE(debut_date)) DESC
"""
df = pd.read_sql_query(query, conn)
df

,playerID,Career (years)
0,altroni01,35.0
1,orourji01,32.0
2,minosmi01,31.0
3,olearch01,30.0
4,lathaar01,29.0
...,...,...
19873,winklbo99,NaN
19874,wrighal99,NaN
19875,yawketo99,NaN
19876,youngni99,NaN


#### 6. What is the distribution of debut months? Hint: Look at the DATE and EXTRACT functions.

In [7]:
query = """
SELECT STRFTIME('%m', debut) AS Month, COUNT(*)
FROM people
GROUP BY Month

"""
df = pd.read_sql_query(query, conn)
df

,Month,COUNT(*)
0,None,196
1,03,92
2,04,4998
3,05,2457
4,06,2155
5,07,2156
6,08,2180
7,09,5336
8,10,308


#### 7. What is the effect of table join order on mean salary for the players listed in the main (master) table? Hint: Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [8]:
# Only keep those who have salary info in salaries
query = """
SELECT people.playerID, people.nameGiven, people.nameFirst, people.nameLast, AVG(salary)
FROM salaries
LEFT JOIN people
ON salaries.playerID = people.playerID
GROUP BY people.playerID

"""
df = pd.read_sql_query(query, conn)
df

,playerID,nameGiven,nameFirst,nameLast,AVG(salary)
0,aardsda01,David Allan,David,Aardsma,1.322821e+06
1,aasedo01,Donald William,Don,Aase,5.750000e+05
2,abadan01,Fausto Andres,Andy,Abad,3.270000e+05
3,abadfe01,Fernando Antonio,Fernando,Abad,7.532800e+05
4,abbotje01,Jeffrey William,Jeff,Abbott,2.462500e+05
...,...,...,...,...,...
5144,zumayjo01,Joel Martin,Joel,Zumaya,7.011667e+05
5145,zuninmi01,Michael Accorsi,Mike,Zunino,5.138000e+05
5146,zupcibo01,Robert,Bob,Zupcic,1.436667e+05
5147,zuvelpa01,Paul,Paul,Zuvella,1.450000e+05


In [9]:
# Keep all the players in people (even without salary info)
query = """
SELECT people.playerID, people.nameGiven, people.nameFirst, people.nameLast, AVG(salary)
FROM people
LEFT JOIN salaries
ON salaries.playerID = people.playerID
GROUP BY people.playerID

"""
df = pd.read_sql_query(query, conn)
df

,playerID,nameGiven,nameFirst,nameLast,AVG(salary)
0,aardsda01,David Allan,David,Aardsma,1.322821e+06
1,aaronha01,Henry Louis,Hank,Aaron,NaN
2,aaronto01,Tommie Lee,Tommie,Aaron,NaN
3,aasedo01,Donald William,Don,Aase,5.750000e+05
4,abadan01,Fausto Andres,Andy,Abad,3.270000e+05
...,...,...,...,...,...
19873,zupofr01,Frank Joseph,Frank,Zupo,NaN
19874,zuvelpa01,Paul,Paul,Zuvella,1.450000e+05
19875,zuverge01,George,George,Zuverink,NaN
19876,zwilldu01,Edward Harrison,Dutch,Zwilling,NaN
